In [ ]:
# Multi-objective evaluation function
def evaluate_knn_multi_cv(individual, X, y, cv_folds=5):
    """Evaluate KNN performance using cross-validation for multiple metrics (precision, recall, f1)"""
    try:
        # For feature selection only
        feature_mask = individual
        k = 5  # Default k
        weights = 'uniform'  # Default weights
        p = 2  # Default p (Euclidean)
        
        # Select features
        selected_features = [i for i, mask in enumerate(feature_mask) if mask > 0.5]
        if len(selected_features) == 0:
            return (0.0, 0.0, 0.0)  # No features selected
        
        X_selected = X.iloc[:, selected_features]
        
        # Ensure k is valid (adjust max k based on dataset size)
        max_k = min(20, len(X_selected) // 5)  # Use at most 1/5 of data size for k
        k = max(1, min(k, max_k))
        
        # Create model
        knn = KNeighborsClassifier(n_neighbors=k, weights=weights, p=p)
        
        cv = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)
        
        # Calculate all three metrics
        precision_scores = cross_val_score(knn, X_selected, y, cv=cv, scoring='precision')
        recall_scores = cross_val_score(knn, X_selected, y, cv=cv, scoring='recall')
        f1_scores = cross_val_score(knn, X_selected, y, cv=cv, scoring='f1')
        
        return (np.mean(precision_scores), np.mean(recall_scores), np.mean(f1_scores))
    
    except Exception as e:
        print(f"Evaluation error: {str(e)}")
        return (0.0, 0.0, 0.0)

# Multi-objective feature selection workflow
def run_ga_workflow_multi(X, y, n_runs=10, generations=20, pop_size=50):
    """Multi-objective feature selection optimization"""
    print(f"Running Multi-Objective Feature Selection (precision, recall, f1)")
    
    results = []
    
    for run in range(n_runs):
        print(f"  Run {run + 1}/{n_runs}")
        
        # Setup GA
        toolbox = base.Toolbox()
        toolbox.register("attr_bool", random.random)
        toolbox.register("individual", tools.initRepeat, creator.MultiIndividual, 
                        toolbox.attr_bool, n=X.shape[1])
        toolbox.register("population", tools.initRepeat, list, toolbox.individual)
        toolbox.register("evaluate", evaluate_knn_multi_cv, X=X, y=y)
        toolbox.register("mate", tools.cxTwoPoint)
        toolbox.register("mutate", tools.mutFlipBit, indpb=0.1)
        toolbox.register("select", tools.selNSGA2)
        
        # Run GA
        population = toolbox.population(n=pop_size)
        hof = tools.ParetoFront()
        
        # Use NSGA-II algorithm
        algorithms.eaMuPlusLambda(population, toolbox, mu=pop_size, lambda_=pop_size,
                               cxpb=0.7, mutpb=0.2, ngen=generations, halloffame=hof, verbose=False)
        
        # Store all non-dominated solutions
        for ind in hof:
            results.append(ind)
    
    return results

# Function to test multi-objective model
def test_multi_model(individual, X_train, y_train, X_test, y_test):
    """Test the multi-objective model on test set and return all metrics"""
    try:
        # Parse individual for feature selection
        feature_mask = individual
        k = 5  # Default k
        weights = 'uniform'  # Default weights
        p = 2  # Default p (Euclidean)
        
        # Select features
        selected_features = [i for i, mask in enumerate(feature_mask) if mask > 0.5]
        if len(selected_features) == 0:
            return {metric: 0.0 for metric in ['accuracy', 'precision', 'recall', 'f1', 'roc_auc', 'pr_auc']}
        
        X_train_selected = X_train.iloc[:, selected_features]
        X_test_selected = X_test.iloc[:, selected_features]
        
        # Train and test model
        knn = KNeighborsClassifier(n_neighbors=k, weights=weights, p=p)
        knn.fit(X_train_selected, y_train)
        y_pred = knn.predict(X_test_selected)
        y_pred_proba = knn.predict_proba(X_test_selected)[:, 1]
        
        # Calculate all metrics
        metrics = {
            'accuracy': accuracy_score(y_test, y_pred),
            'precision': precision_score(y_test, y_pred, zero_division=0),
            'recall': recall_score(y_test, y_pred, zero_division=0),
            'f1': f1_score(y_test, y_pred, zero_division=0),
            'roc_auc': roc_auc_score(y_test, y_pred_proba),
            'pr_auc': average_precision_score(y_test, y_pred_proba),
            'selected_features': len(selected_features)
        }
        
        return metrics
    
    except Exception as e:
        print(f"Test error: {str(e)}")
        return {metric: 0.0 for metric in ['accuracy', 'precision', 'recall', 'f1', 'roc_auc', 'pr_auc', 'selected_features']}

In [ ]:
# KNN Genetic Algorithm Optimization for Promise Datasets
# Complete implementation with 4 GA workflows and hyperparameter display

# Install required libraries
!pip install pandas numpy scikit-learn imbalanced-learn deap shap openpyxl matplotlib seaborn scipy

# Import all required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (
    precision_score, recall_score, f1_score, accuracy_score, 
    roc_auc_score, average_precision_score, classification_report
)
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from deap import base, creator, tools, algorithms
from scipy.io import arff
import random
import warnings
import io
from io import StringIO
from google.colab import files
from collections import defaultdict
import statistics

warnings.filterwarnings('ignore')
np.random.seed(42)
random.seed(42)

print("All libraries installed and imported successfully!")

# Create DEAP fitness and individual classes
creator.create("FitnessMax", base.Fitness, weights=(1.0,))  # Maximize fitness
creator.create("Individual", list, fitness=creator.FitnessMax)

# Data upload and preprocessing functions
def upload_and_load_files():
    """Upload CSV or ARFF files and return dictionary of dataframes"""
    print("Please upload your Promise dataset files (CSV or ARFF format)...")
    uploaded = files.upload()
    
    datasets = {}
    for filename in uploaded.keys():
        if filename.endswith('.csv'):
            df = pd.read_csv(io.BytesIO(uploaded[filename]))
            datasets[filename.replace('.csv', '')] = df
            print(f"Loaded CSV {filename}: {df.shape}")
        elif filename.endswith('.arff'):
            # Load ARFF file - fix for bytes-like object error
            try:
                # First try with default loading (binary mode)
                data, meta = arff.loadarff(io.BytesIO(uploaded[filename]))
                df = pd.DataFrame(data)
            except TypeError:
                # If we get a TypeError about bytes-like object, decode to string first
                arff_content = uploaded[filename].decode('utf-8')
                data, meta = arff.loadarff(io.StringIO(arff_content))
                df = pd.DataFrame(data)
            
            # Convert byte strings to regular strings for object columns
            for col in df.columns:
                if df[col].dtype == 'object':
                    try:
                        df[col] = df[col].str.decode('utf-8')
                    except AttributeError:
                        # If it's not bytes, leave as is
                        pass
            
            datasets[filename.replace('.arff', '')] = df
            print(f"Loaded ARFF {filename}: {df.shape}")
            print(f"Attributes: {list(meta.names())}")
    
    return datasets

def preprocess_dataset(df, target_column=None):
    """Clean, encode, and prepare dataset for ML"""
    df = df.copy()
    
    # Handle missing values
    numeric_columns = df.select_dtypes(include=[np.number]).columns
    categorical_columns = df.select_dtypes(include=['object']).columns
    
    # Impute missing values
    if len(numeric_columns) > 0:
        num_imputer = SimpleImputer(strategy='median')
        df[numeric_columns] = num_imputer.fit_transform(df[numeric_columns])
    
    if len(categorical_columns) > 0:
        cat_imputer = SimpleImputer(strategy='most_frequent')
        df[categorical_columns] = cat_imputer.fit_transform(df[categorical_columns])
    
    # Handle target column - try common names if not specified
    if target_column is None:
        # Common target column names in Promise datasets (bug is now first priority)
        possible_targets = ['bug', 'defects', 'class', 'defective', 'Class']
        target_column = None
        
        for col in possible_targets:
            if col in df.columns:
                target_column = col
                break
        
        if target_column is None:
            # Ask user to specify target column
            print(f"Available columns: {list(df.columns)}")
            target_column = input("Please specify the target column name: ")
    
    if target_column in df.columns:
        y = df[target_column]
        X = df.drop(columns=[target_column])
        print(f"Using '{target_column}' as target column")
    else:
        # Assume last column is target
        y = df.iloc[:, -1]
        X = df.iloc[:, :-1]
        print(f"Using last column '{df.columns[-1]}' as target column")
    
    # Encode categorical features
    categorical_features = X.select_dtypes(include=['object']).columns
    label_encoders = {}
    
    for col in categorical_features:
        le = LabelEncoder()
        X[col] = le.fit_transform(X[col].astype(str))
        label_encoders[col] = le
    
    # Encode target if categorical
    if y.dtype == 'object' or y.dtype == 'bool':
        target_encoder = LabelEncoder()
        y = target_encoder.fit_transform(y)
        print(f"Target classes: {list(target_encoder.classes_)}")
        print(f"Target encoding: false=0, true=1")
    else:
        target_encoder = None
    
    # Scale features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    X_scaled = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)
    
    return X_scaled, y, scaler, label_encoders, target_encoder

def apply_smote(X, y, random_state=42):
    """Apply SMOTE to balance the dataset"""
    smote = SMOTE(random_state=random_state)
    X_balanced, y_balanced = smote.fit_resample(X, y)
    
    print(f"Original class distribution: {np.bincount(y)}")
    print(f"Balanced class distribution: {np.bincount(y_balanced)}")
    
    return X_balanced, y_balanced

# Hyperparameter decoding and display functions
def decode_hyperparameters(individual, workflow_type, feature_names=None):
    """Decode individual to human-readable hyperparameters"""
    result = {}
    
    if workflow_type == 'features_only':
        feature_mask = individual
        result['k'] = 5  # Default
        result['weights'] = 'uniform'  # Default
        result['distance_metric'] = 'euclidean'  # Default
        if feature_names:
            selected_features = [feature_names[i] for i, mask in enumerate(feature_mask) if mask > 0.5]
            result['selected_features'] = selected_features
            result['num_features'] = len(selected_features)
    elif workflow_type == 'params_only':
        k = individual[0]
        weights = 'distance' if individual[1] > 0.5 else 'uniform'
        p = 1 if individual[2] < 0.33 else (2 if individual[2] < 0.66 else 3)
        
        result['k'] = int(k * 20) if isinstance(k, float) else k
        result['weights'] = weights
        result['distance_metric'] = 'manhattan' if p == 1 else ('euclidean' if p == 2 else 'minkowski')
        result['num_features'] = 'all'
        if feature_names:
            result['selected_features'] = feature_names
    else:  # joint or sequential
        feature_mask = individual[:-3]
        k = individual[-3]
        weights = 'distance' if individual[-2] > 0.5 else 'uniform'
        p = 1 if individual[-1] < 0.33 else (2 if individual[-1] < 0.66 else 3)
        
        result['k'] = int(k * 20) if isinstance(k, float) else k
        result['weights'] = weights
        result['distance_metric'] = 'manhattan' if p == 1 else ('euclidean' if p == 2 else 'minkowski')
        if feature_names:
            selected_features = [feature_names[i] for i, mask in enumerate(feature_mask) if mask > 0.5]
            result['selected_features'] = selected_features
            result['num_features'] = len(selected_features)
    
    return result

def display_best_hyperparameters(best_individuals, workflow_names, metrics, feature_names=None):
    """Display best hyperparameters for each workflow and metric combination"""
    print("\n" + "="*80)
    print("BEST HYPERPARAMETERS BY WORKFLOW AND OPTIMIZATION METRIC")
    print("="*80)
    
    for metric in metrics:
        print(f"\n🎯 OPTIMIZATION METRIC: {metric.upper()}")
        print("-" * 60)
        
        for workflow_name in workflow_names:
            workflow_type = workflow_name.split('_', 1)[1]
            
            if (metric, workflow_name) in best_individuals:
                individuals = best_individuals[(metric, workflow_name)]
                
                print(f"\n📊 Workflow: {workflow_name.replace('_', ' ').title()}")
                
                # Decode all individuals
                decoded_params = []
                for individual in individuals:
                    params = decode_hyperparameters(individual, workflow_type, feature_names)
                    decoded_params.append(params)
                
                # Calculate statistics for hyperparameters
                if decoded_params:
                    # K values
                    k_values = [p['k'] for p in decoded_params]
                    k_mean = statistics.mean(k_values)
                    k_mode = statistics.mode(k_values) if k_values else k_values[0]
                    
                    # Weights
                    weights_values = [p['weights'] for p in decoded_params]
                    weights_mode = statistics.mode(weights_values)
                    
                    # Distance metrics
                    distance_values = [p['distance_metric'] for p in decoded_params]
                    distance_mode = statistics.mode(distance_values)
                    
                    print(f"  • K neighbors: {k_mode} (mode), {k_mean:.1f} (mean), range: {min(k_values)}-{max(k_values)}")
                    print(f"  • Weight scheme: {weights_mode}")
                    print(f"  • Distance metric: {distance_mode}")
                    
                    # Feature selection statistics
                    if workflow_type != 'params_only':
                        num_features = [p['num_features'] for p in decoded_params]
                        if num_features:
                            feat_mean = statistics.mean(num_features)
                            feat_mode = statistics.mode(num_features)
                            print(f"  • Features selected: {feat_mode} (mode), {feat_mean:.1f} (mean), range: {min(num_features)}-{max(num_features)}")
                            
                            # Most frequently selected features
                            if feature_names and workflow_type != 'params_only':
                                feature_counts = defaultdict(int)
                                for p in decoded_params:
                                    if 'selected_features' in p:
                                        for feat in p['selected_features']:
                                            feature_counts[feat] += 1
                                
                                if feature_counts:
                                    top_features = sorted(feature_counts.items(), key=lambda x: x[1], reverse=True)[:5]
                                    print(f"  • Top selected features:")
                                    for feat, count in top_features:
                                        percentage = (count / len(decoded_params)) * 100
                                        print(f"    - {feat}: {count}/{len(decoded_params)} runs ({percentage:.1f}%)")
                    else:
                        print(f"  • Features: All features used")
    
    print("\n" + "="*80)

# Evaluation functions
def evaluate_knn_cv(individual, X, y, metric='f1', cv_folds=5):
    """Evaluate KNN performance using cross-validation"""
    try:
        # Parse individual based on workflow type
        if len(individual) == X.shape[1]:  # Feature selection only
            feature_mask = individual
            k = 5  # Default k
            weights = 'uniform'  # Default weights
            p = 2  # Default p (Euclidean)
        elif len(individual) == 3:  # Hyperparameters only
            feature_mask = [1] * X.shape[1]  # Use all features
            k = individual[0]
            weights = 'distance' if individual[1] > 0.5 else 'uniform'
            p = 1 if individual[2] < 0.33 else (2 if individual[2] < 0.66 else 3)
        else:  # Joint optimization
            feature_mask = individual[:-3]
            k = individual[-3]
            weights = 'distance' if individual[-2] > 0.5 else 'uniform'
            p = 1 if individual[-1] < 0.33 else (2 if individual[-1] < 0.66 else 3)
        
        # Select features
        selected_features = [i for i, mask in enumerate(feature_mask) if mask > 0.5]
        if len(selected_features) == 0:
            return (0.0,)  # No features selected
        
        X_selected = X.iloc[:, selected_features]
        
        # Ensure k is valid (adjust max k based on dataset size)
        max_k = min(20, len(X_selected) // 5)  # Use at most 1/5 of data size for k
        if isinstance(k, float):
            k = max(1, min(int(k * max_k), max_k))
        else:
            k = max(1, min(k, max_k))
        
        # Create and evaluate model
        knn = KNeighborsClassifier(n_neighbors=k, weights=weights, p=p)
        
        cv = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)
        
        if metric == 'accuracy':
            scores = cross_val_score(knn, X_selected, y, cv=cv, scoring='accuracy')
        elif metric == 'precision':
            scores = cross_val_score(knn, X_selected, y, cv=cv, scoring='precision')
        elif metric == 'recall':
            scores = cross_val_score(knn, X_selected, y, cv=cv, scoring='recall')
        elif metric == 'f1':
            scores = cross_val_score(knn, X_selected, y, cv=cv, scoring='f1')
        elif metric == 'roc_auc':
            scores = cross_val_score(knn, X_selected, y, cv=cv, scoring='roc_auc')
        elif metric == 'pr_auc':
            scores = cross_val_score(knn, X_selected, y, cv=cv, scoring='average_precision')
        
        return (np.mean(scores),)
    
    except Exception as e:
        return (0.0,)

def test_final_model(individual, X_train, y_train, X_test, y_test, workflow_type):
    """Test the final model on test set and return all metrics"""
    try:
        # Parse individual based on workflow type
        if workflow_type == 'features_only':
            feature_mask = individual
            k = 5
            weights = 'uniform'
            p = 2
        elif workflow_type == 'params_only':
            feature_mask = [1] * X_train.shape[1]
            k = individual[0]
            weights = 'distance' if individual[1] > 0.5 else 'uniform'
            p = 1 if individual[2] < 0.33 else (2 if individual[2] < 0.66 else 3)
        else:  # joint or sequential
            feature_mask = individual[:-3]
            k = individual[-3]
            weights = 'distance' if individual[-2] > 0.5 else 'uniform'
            p = 1 if individual[-1] < 0.33 else (2 if individual[-1] < 0.66 else 3)
        
        # Select features
        selected_features = [i for i, mask in enumerate(feature_mask) if mask > 0.5]
        if len(selected_features) == 0:
            return {metric: 0.0 for metric in ['accuracy', 'precision', 'recall', 'f1', 'roc_auc', 'pr_auc']}
        
        X_train_selected = X_train.iloc[:, selected_features]
        X_test_selected = X_test.iloc[:, selected_features]
        
        # Ensure k is valid
        if isinstance(k, float):
            k = max(1, min(int(k * 20), min(20, len(X_train_selected) - 1)))
        else:
            k = max(1, min(k, min(20, len(X_train_selected) - 1)))
        
        # Train and test model
        knn = KNeighborsClassifier(n_neighbors=k, weights=weights, p=p)
        knn.fit(X_train_selected, y_train)
        y_pred = knn.predict(X_test_selected)
        y_pred_proba = knn.predict_proba(X_test_selected)[:, 1]
        
        # Calculate all metrics
        metrics = {
            'accuracy': accuracy_score(y_test, y_pred),
            'precision': precision_score(y_test, y_pred, zero_division=0),
            'recall': recall_score(y_test, y_pred, zero_division=0),
            'f1': f1_score(y_test, y_pred, zero_division=0),
            'roc_auc': roc_auc_score(y_test, y_pred_proba),
            'pr_auc': average_precision_score(y_test, y_pred_proba)
        }
        
        return metrics
    
    except Exception as e:
        return {metric: 0.0 for metric in ['accuracy', 'precision', 'recall', 'f1', 'roc_auc', 'pr_auc']}

# GA Workflow implementations
def run_ga_workflow_a(X, y, metric='f1', n_runs=10, generations=20, pop_size=50):
    """Workflow A: Feature selection only"""
    print(f"Running Workflow A: Feature Selection Only ({metric})")
    
    results = []
    
    for run in range(n_runs):
        print(f"  Run {run + 1}/{n_runs}")
        
        # Setup GA
        toolbox = base.Toolbox()
        toolbox.register("attr_bool", random.random)
        toolbox.register("individual", tools.initRepeat, creator.Individual, 
                        toolbox.attr_bool, n=X.shape[1])
        toolbox.register("population", tools.initRepeat, list, toolbox.individual)
        toolbox.register("evaluate", evaluate_knn_cv, X=X, y=y, metric=metric)
        toolbox.register("mate", tools.cxTwoPoint)
        toolbox.register("mutate", tools.mutFlipBit, indpb=0.1)
        toolbox.register("select", tools.selTournament, tournsize=3)
        
        # Run GA
        population = toolbox.population(n=pop_size)
        hof = tools.HallOfFame(1)
        
        algorithms.eaSimple(population, toolbox, cxpb=0.7, mutpb=0.2, 
                           ngen=generations, halloffame=hof, verbose=False)
        
        results.append(hof[0])
    
    return results

def run_ga_workflow_b(X, y, metric='f1', n_runs=10, generations=20, pop_size=50):
    """Workflow B: Hyperparameter tuning only"""
    print(f"Running Workflow B: Hyperparameter Tuning Only ({metric})")
    
    results = []
    
    for run in range(n_runs):
        print(f"  Run {run + 1}/{n_runs}")
        
        # Setup GA for hyperparameters: [k, weights, p]
        toolbox = base.Toolbox()
        toolbox.register("attr_k", random.randint, 1, 20)
        toolbox.register("attr_weights", random.random)
        toolbox.register("attr_p", random.random)
        toolbox.register("individual", tools.initCycle, creator.Individual,
                        (toolbox.attr_k, toolbox.attr_weights, toolbox.attr_p))
        toolbox.register("population", tools.initRepeat, list, toolbox.individual)
        toolbox.register("evaluate", evaluate_knn_cv, X=X, y=y, metric=metric)
        toolbox.register("mate", tools.cxBlend, alpha=0.3)
        toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.2, indpb=0.3)
        toolbox.register("select", tools.selTournament, tournsize=3)
        
        # Run GA
        population = toolbox.population(n=pop_size)
        hof = tools.HallOfFame(1)
        
        algorithms.eaSimple(population, toolbox, cxpb=0.7, mutpb=0.2,
                           ngen=generations, halloffame=hof, verbose=False)
        
        results.append(hof[0])
    
    return results

def run_ga_workflow_c(X, y, metric='f1', n_runs=10, generations=20, pop_size=50):
    """Workflow C: Joint feature selection + hyperparameter tuning"""
    print(f"Running Workflow C: Joint Optimization ({metric})")
    
    results = []
    
    for run in range(n_runs):
        print(f"  Run {run + 1}/{n_runs}")
        
        # Setup GA for joint optimization: [features..., k, weights, p]
        toolbox = base.Toolbox()
        
        def create_individual():
            # Feature mask + hyperparameters
            features = [random.random() for _ in range(X.shape[1])]
            hyperparams = [random.randint(1, 20), random.random(), random.random()]
            return creator.Individual(features + hyperparams)
        
        toolbox.register("individual", create_individual)
        toolbox.register("population", tools.initRepeat, list, toolbox.individual)
        toolbox.register("evaluate", evaluate_knn_cv, X=X, y=y, metric=metric)
        toolbox.register("mate", tools.cxTwoPoint)
        toolbox.register("mutate", tools.mutFlipBit, indpb=0.1)
        toolbox.register("select", tools.selTournament, tournsize=3)
        
        # Run GA
        population = toolbox.population(n=pop_size)
        hof = tools.HallOfFame(1)
        
        algorithms.eaSimple(population, toolbox, cxpb=0.7, mutpb=0.2,
                           ngen=generations, halloffame=hof, verbose=False)
        
        results.append(hof[0])
    
    return results

def run_ga_workflow_d(X, y, metric='f1', n_runs=10, generations=20, pop_size=50):
    """Workflow D: Sequential hyperparameter tuning then feature selection"""
    print(f"Running Workflow D: Sequential Optimization ({metric})")
    
    results = []
    
    for run in range(n_runs):
        print(f"  Run {run + 1}/{n_runs}")
        
        # Phase 1: Optimize hyperparameters
        toolbox1 = base.Toolbox()
        toolbox1.register("attr_k", random.randint, 1, 20)
        toolbox1.register("attr_weights", random.random)
        toolbox1.register("attr_p", random.random)
        toolbox1.register("individual", tools.initCycle, creator.Individual,
                         (toolbox1.attr_k, toolbox1.attr_weights, toolbox1.attr_p))
        toolbox1.register("population", tools.initRepeat, list, toolbox1.individual)
        toolbox1.register("evaluate", evaluate_knn_cv, X=X, y=y, metric=metric)
        toolbox1.register("mate", tools.cxBlend, alpha=0.3)
        toolbox1.register("mutate", tools.mutGaussian, mu=0, sigma=0.2, indpb=0.3)
        toolbox1.register("select", tools.selTournament, tournsize=3)
        
        population1 = toolbox1.population(n=pop_size)
        hof1 = tools.HallOfFame(1)
        
        algorithms.eaSimple(population1, toolbox1, cxpb=0.7, mutpb=0.2,
                           ngen=generations//2, halloffame=hof1, verbose=False)
        
        best_hyperparams = hof1[0]
        
        # Phase 2: Optimize features with fixed hyperparameters
        def create_individual_phase2():
            features = [random.random() for _ in range(X.shape[1])]
            return creator.Individual(features + list(best_hyperparams))
        
        toolbox2 = base.Toolbox()
        toolbox2.register("individual", create_individual_phase2)
        toolbox2.register("population", tools.initRepeat, list, toolbox2.individual)
        toolbox2.register("evaluate", evaluate_knn_cv, X=X, y=y, metric=metric)
        toolbox2.register("mate", tools.cxTwoPoint)
        toolbox2.register("mutate", tools.mutFlipBit, indpb=0.1)
        toolbox2.register("select", tools.selTournament, tournsize=3)
        
        population2 = toolbox2.population(n=pop_size)
        hof2 = tools.HallOfFame(1)
        
        algorithms.eaSimple(population2, toolbox2, cxpb=0.7, mutpb=0.2,
                           ngen=generations//2, halloffame=hof2, verbose=False)
        
        results.append(hof2[0])
    
    return results

# Main pipeline functions
def main_pipeline():
    """Main execution pipeline"""
    # Step 1: Upload and load datasets
    print("=== Step 1: Upload Dataset Files ===")
    datasets = upload_and_load_files()
    
    if not datasets:
        print("No datasets uploaded. Please restart and upload dataset files.")
        return
    
    # Step 2: Select train and test versions
    print("\n=== Step 2: Select Train and Test Versions ===")
    print("Available datasets:")
    for i, name in enumerate(datasets.keys()):
        print(f"{i}: {name}")
    
    train_idx = int(input("Select train dataset index: "))
    train_name = list(datasets.keys())[train_idx]
    train_df = datasets[train_name]
    
    test_indices = input("Select test dataset indices (comma-separated): ")
    test_indices = [int(x.strip()) for x in test_indices.split(',')]
    test_names = [list(datasets.keys())[i] for i in test_indices]
    test_dfs = [datasets[name] for name in test_names]
    
    print(f"Train dataset: {train_name}")
    print(f"Test datasets: {test_names}")
    
    # Step 3: Preprocess datasets
    print("\n=== Step 3: Preprocessing ===")
    
    # Preprocess train data
    X_train, y_train, scaler, label_encoders, target_encoder = preprocess_dataset(train_df)
    X_train_balanced, y_train_balanced = apply_smote(X_train, y_train)
    
    # Store feature names for hyperparameter display
    feature_names = list(X_train.columns)
    
    # Preprocess test data
    test_data = []
    for test_df, test_name in zip(test_dfs, test_names):
        X_test, y_test, _, _, _ = preprocess_dataset(test_df)
        # Apply same scaling as train data
        X_test_scaled = pd.DataFrame(scaler.transform(X_test), 
                                   columns=X_test.columns, index=X_test.index)
        test_data.append((X_test_scaled, y_test, test_name))
    
    print(f"Train data shape: {X_train_balanced.shape}")
    print(f"Number of test datasets: {len(test_data)}")
    
    # Step 4: Run GA workflows
    print("\n=== Step 4: Running GA Workflows ===")
    
    metrics = ['precision', 'recall', 'f1', 'accuracy', 'roc_auc', 'pr_auc']
    workflows = {
        'A_features_only': run_ga_workflow_a,
        'B_params_only': run_ga_workflow_b,
        'C_joint': run_ga_workflow_c,
        'D_sequential': run_ga_workflow_d
    }
    
    all_results = {}
    best_individuals = {}  # Store best individuals for hyperparameter display
    
    for metric in metrics:
        print(f"\n--- Optimizing for {metric.upper()} ---")
        
        for workflow_name, workflow_func in workflows.items():
            # Run GA optimization
            ga_results = workflow_func(X_train_balanced, y_train_balanced, 
                                     metric=metric, n_runs=10, generations=20, pop_size=50)
            
            # Store best individuals for hyperparameter display
            best_individuals[(metric, workflow_name)] = ga_results
            
            # Test each best model on all test sets
            workflow_type = workflow_name.split('_', 1)[1]
            
            for run_idx, best_individual in enumerate(ga_results):
                for X_test, y_test, test_name in test_data:
                    test_metrics = test_final_model(best_individual, 
                                                  X_train_balanced, y_train_balanced,
                                                  X_test, y_test, workflow_type)
                    
                    key = (metric, workflow_name, test_name, run_idx)
                    all_results[key] = test_metrics
    
    # Display best hyperparameters
    display_best_hyperparameters(best_individuals, list(workflows.keys()), metrics, feature_names)
    
    return all_results, test_names, metrics

def aggregate_and_save_results(all_results, test_names, metrics):
    """Aggregate results and save to Excel"""
    print("\n=== Step 5: Aggregating Results ===")
    
    workflows = ['A_features_only', 'B_params_only', 'C_joint', 'D_sequential']
    
    # Create results DataFrame
    results_data = []
    
    for opt_metric in metrics:
        for workflow in workflows:
            for test_name in test_names:
                # Collect all runs for this combination
                run_results = {metric: [] for metric in metrics}
                
                for run_idx in range(10):
                    key = (opt_metric, workflow, test_name, run_idx)
                    if key in all_results:
                        for metric in metrics:
                            run_results[metric].append(all_results[key][metric])
                
                # Calculate mean and std for each metric
                row = {
                    'Optimization_Metric': opt_metric,
                    'Workflow': workflow,
                    'Test_Dataset': test_name
                }
                
                for metric in metrics:
                    if run_results[metric]:
                        mean_val = np.mean(run_results[metric])
                        std_val = np.std(run_results[metric])
                        row[f'{metric}_mean'] = mean_val
                        row[f'{metric}_std'] = std_val
                        row[f'{metric}_mean_std'] = f"{mean_val:.4f}±{std_val:.4f}"
                    else:
                        row[f'{metric}_mean'] = 0.0
                        row[f'{metric}_std'] = 0.0
                        row[f'{metric}_mean_std'] = "0.0000±0.0000"
                
                results_data.append(row)
    
    results_df = pd.DataFrame(results_data)
    
    # Save to Excel
    results_df.to_excel('results.xlsx', index=False)
    
    print(f"Results saved to results.xlsx")
    print(f"Results shape: {results_df.shape}")
    
    # Display summary
    print("\nSample results:")
    display_columns = ['Optimization_Metric', 'Workflow', 'Test_Dataset', 'f1_mean_std', 'accuracy_mean_std']
    print(results_df[display_columns].head(10))
    
    # Download file
    files.download('results.xlsx')
    
    return results_df

def analyze_results():
    """Analyze and visualize the results"""
    try:
        results_df = pd.read_excel('results.xlsx')
        
        print("\n=== RESULTS ANALYSIS ===")
        print(f"Total combinations: {len(results_df)}")
        print(f"Workflows: {results_df['Workflow'].unique()}")
        print(f"Optimization metrics: {results_df['Optimization_Metric'].unique()}")
        print(f"Test datasets: {results_df['Test_Dataset'].unique()}")
        
        # Create summary plots
        plt.figure(figsize=(15, 10))
        
        # Plot 1: F1 scores by workflow and optimization metric
        plt.subplot(2, 2, 1)
        workflow_f1 = results_df.groupby(['Workflow', 'Optimization_Metric'])['f1_mean'].mean().unstack()
        sns.heatmap(workflow_f1, annot=True, fmt='.3f', cmap='viridis')
        plt.title('Average F1 Score by Workflow and Optimization Metric')
        plt.ylabel('Workflow')
        
        # Plot 2: Accuracy scores by workflow and optimization metric
        plt.subplot(2, 2, 2)
        workflow_acc = results_df.groupby(['Workflow', 'Optimization_Metric'])['accuracy_mean'].mean().unstack()
        sns.heatmap(workflow_acc, annot=True, fmt='.3f', cmap='viridis')
        plt.title('Average Accuracy by Workflow and Optimization Metric')
        plt.ylabel('Workflow')
        
        # Plot 3: ROC-AUC scores by workflow
        plt.subplot(2, 2, 3)
        workflow_roc = results_df.groupby('Workflow')['roc_auc_mean'].mean()
        workflow_roc.plot(kind='bar')
        plt.title('Average ROC-AUC by Workflow')
        plt.ylabel('ROC-AUC')
        plt.xticks(rotation=45)
        
        # Plot 4: PR-AUC scores by workflow
        plt.subplot(2, 2, 4)
        workflow_pr = results_df.groupby('Workflow')['pr_auc_mean'].mean()
        workflow_pr.plot(kind='bar')
        plt.title('Average PR-AUC by Workflow')
        plt.ylabel('PR-AUC')
        plt.xticks(rotation=45)
        
        plt.tight_layout()
        plt.show()
        
        # Best performing combinations
        print("\nTop 10 combinations by F1 score:")
        top_f1 = results_df.nlargest(10, 'f1_mean')[['Optimization_Metric', 'Workflow', 'Test_Dataset', 'f1_mean_std']]
        print(top_f1.to_string(index=False))
        
        print("\nTop 10 combinations by Accuracy:")
        top_acc = results_df.nlargest(10, 'accuracy_mean')[['Optimization_Metric', 'Workflow', 'Test_Dataset', 'accuracy_mean_std']]
        print(top_acc.to_string(index=False))
        
    except FileNotFoundError:
        print("Results file not found. Please run the main pipeline first.")
    except Exception as e:
        print(f"Error analyzing results: {str(e)}")

# Execute the complete pipeline
if __name__ == "__main__":
    print("🚀 Starting KNN Genetic Algorithm Optimization Pipeline...")
    print("="*80)
    
    try:
        # Run main pipeline
        all_results, test_names, metrics = main_pipeline()
        
        # Aggregate and save results
        final_results = aggregate_and_save_results(all_results, test_names, metrics)
        
        # Analyze results
        analyze_results()
        
        print("\n" + "="*80)
        print("🎉 PIPELINE COMPLETED SUCCESSFULLY!")
        print("📊 Results have been saved to 'results.xlsx' and downloaded.")
        print("📈 Best hyperparameters displayed above.")
        print("📋 Analysis plots and top combinations shown.")
        
    except Exception as e:
        print(f"❌ Error during execution: {str(e)}")
        import traceback
        traceback.print_exc()

print("\n📋 USAGE INSTRUCTIONS:")
print("1. Upload your Promise dataset files (CSV or ARFF format)")
print("2. Select train and test datasets when prompted")
print("3. Wait for the complete analysis (may take 30-60 minutes)")
print("4. Review hyperparameters, results, and analysis")
print("5. Download the generated Excel file with detailed results")